In [1]:
from zipline.pipeline import Pipeline
from zipline.component.research import run_pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

## Custom Factors
When we first looked at factors, we explored the set of built-in factors. Frequently, a desired computation isn't included as a built-in factor. One of the most powerful features of the Pipeline API is that it allows us to define our own custom factors. When a desired computation doesn't exist as a built-in, we define a custom factor.

Conceptually, a custom factor is identical to a built-in factor. It accepts `inputs`, `window_length`, and `mask` as constructor arguments, and returns a `Factor` object each day.

Let's take an example of a computation that doesn't exist as a built-in: standard deviation. To create a factor that computes the [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation) over a trailing window, we can subclass `zipline.pipeline.CustomFactor` and implement a compute method whose signature is:


```
def compute(self, today, asset_ids, out, *inputs):
    ...
```

- `*inputs` are M x N [numpy arrays](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html), where M is the `window_length` and N is the number of securities (usually around ~8000 unless a `mask` is provided). `*inputs` are trailing data windows. Note that there will be one M x N array for each `BoundColumn` provided in the factor's `inputs` list. The data type of each array will be the `dtype` of the corresponding `BoundColumn`.
- `out` is an empty array of length N. `out` will be the output of our custom factor each day. The job of compute is to write output values into out.
- `asset_ids` will be an integer [array](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.array.html) of length N containing security ids corresponding to the columns in our `*inputs` arrays.
- `today` will be a [pandas Timestamp](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#converting-to-timestamps) representing the day for which `compute` is being called.

Of these, `*inputs` and `out` are most commonly used.

An instance of `CustomFactor` that’s been added to a pipeline will have its compute method called every day. For example, let's define a custom factor that computes the standard deviation of the close price over the last 5 days. To start, let's add `CustomFactor` and `numpy` to our import statements.

In [2]:
from zipline.pipeline import CustomFactor
import numpy

Next, let's define our custom factor to calculate the standard deviation over a trailing window using [numpy.nanstd](http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.nanstd.html):

In [3]:
class StdDev(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        # Calculates the column-wise standard deviation, ignoring NaNs
        out[:] = numpy.nanstd(values, axis=0)

Finally, let's instantiate our factor in `make_pipeline()`:

In [4]:
def make_pipeline():
    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev
        }
    )

When this pipeline is run, `StdDev.compute()` will be called every day with data as follows:

- `values`: An M x N [numpy array](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html), where M is 5 (`window_length`), and N is ~8000 (the number of securities in our database on the day in question).
- `out`: An empty array of length N (~8000). In this example, the job of `compute` is to populate `out` with an array storing of 5-day close price standard deviations.

In [5]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

C:\Users\liude\Anaconda3\envs\bt36\lib\site-packages\numpy\lib\nanfunctions.py:1427: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


std_dev
2015-05-05 00:00:00+00:00 Equity(000001 [平安银行])  1.107068e-01
                          Equity(000002 [万 科Ａ])  2.032339e-01
                          Equity(000004 [国农科技])  5.560719e-01
                          Equity(000005 [世纪星源])  0.000000e+00
                          Equity(000006 [深振业Ａ])  2.758985e-01
                          Equity(000007 [全新好])   1.047924e+00
                          Equity(000008 [神州高铁])  3.552714e-15
                          Equity(000009 [中国宝安])  9.867725e-01
                          Equity(000010 [美丽生态])  0.000000e+00
                          Equity(000011 [深物业A])  2.065914e-01
                          Equity(000012 [南 玻Ａ])  4.748642e-01
                          Equity(000014 [沙河股份])  2.818510e-01
                          Equity(000016 [深康佳Ａ])  1.181333e+00
                          Equity(000017 [深中华A])  6.327274e-01
                          Equity(000018 [神州长城])  9.822016e-01
                          Equity(000019 [深深宝Ａ])  6.080329e-01
                          Equity(000020 [深华发Ａ])  4.770880e-01
                          Equity(000021 [深科技])   3.064245e-01
                          Equity(000022 [深赤湾Ａ])  3.552714e-15
                          Equity(000023 [深天地Ａ])  6.072561e-01
                          Equity(000024 [招商地产])  0.000000e+00
                          Equity(000025 [特 力Ａ])  5.966574e-01
                          Equity(000026 [飞亚达Ａ])  2.854750e-01
                          Equity(000027 [深圳能源])  2.844222e-01
                          Equity(000028 [国药一致])  2.044323e+00
                          Equity(000029 [深深房Ａ])  2.861748e-01
                          Equity(000030 [富奥股份])  3.313367e-01
                          Equity(000031 [中粮地产])  3.155693e-01
                          Equity(000032 [深桑达Ａ])  4.675468e-01
                          Equity(000033 [新都退])   3.850195e-01
...                                                       ...
                          Equity(603609 [禾丰牧业])  3.805575e-01
                          Equity(603611 [诺力股份])  8.729353e-01
                          Equity(603618 [杭电股份])  1.286568e+00
                          Equity(603636 [南威软件])  1.549729e+00
                          Equity(603678 [火炬电子])  1.942318e+00
                          Equity(603686 [龙马环卫])  4.043309e+00
                          Equity(603688 [石英股份])  4.248011e-01
                          Equity(603698 [航天工程])  7.044147e-01
                          Equity(603699 [纽威股份])  4.604780e-01
                          Equity(603703 [盛洋科技])  3.240957e+00
                          Equity(603729 [龙韵股份])  3.064959e+00
                          Equity(603766 [隆鑫通用])  6.641385e-01
                          Equity(603788 [宁波高发])  4.212648e-01
                          Equity(603789 [星光农机])  2.657205e+00
                          Equity(603799 [华友钴业])  4.247070e-01
                          Equity(603806 [福斯特])   1.082098e+00
                          Equity(603808 [歌力思])   4.458692e+00
                          Equity(603818 [曲美家居])  2.823013e+00
                          Equity(603828 [柯利达])   6.668853e-01
                          Equity(603869 [北部湾旅])  1.039988e+00
                          Equity(603883 [老百姓])   4.698385e+00
                          Equity(603889 [新澳股份])  4.534578e-01
                          Equity(603898 [好莱客])   2.119343e+00
                          Equity(603899 [晨光文具])  9.667802e-01
                          Equity(603939 [益丰药房])  1.892729e+00
                          Equity(603969 [银龙股份])  5.426380e-01
                          Equity(603988 [中电电机])  8.323605e-01
                          Equity(603993 [洛阳钼业])  5.678944e-01
                          Equity(603997 [继峰股份])  1.773584e-01
                          Equity(603998 [方盛制药])  7.682448e-01

[2702 rows x 1 columns]

### Default Inputs
When writing a custom factor, we can set default `inputs` and `window_length` in our `CustomFactor` subclass. For example, let's define the `TenDayMeanDifference` custom factor to compute the mean difference between two data columns over a trailing window using [numpy.nanmean](http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.nanmean.html). Let's set the default `inputs` to `[USEquityPricing.close, USEquityPricing.open]` and the default `window_length` to 10:

In [6]:
class TenDayMeanDifference(CustomFactor):
    # Default inputs.
    inputs = [USEquityPricing.close, USEquityPricing.open]
    window_length = 10
    def compute(self, today, asset_ids, out, close, open):
        # Calculates the column-wise mean difference, ignoring NaNs
        out[:] = numpy.nanmean(close - open, axis=0)

<i>Remember in this case that `close` and `open` are each 10 x ~8000 2D [numpy arrays.](http://docs.scipy.org/doc/numpy-1.10.1/reference/arrays.ndarray.html)</i>

If we call `TenDayMeanDifference` without providing any arguments, it will use the defaults.

In [7]:
# Computes the 10-day mean difference between the daily open and close prices.
close_open_diff = TenDayMeanDifference()

The defaults can be manually overridden by specifying arguments in the constructor call.

In [8]:
# Computes the 10-day mean difference between the daily high and low prices.
high_low_diff = TenDayMeanDifference(inputs=[USEquityPricing.high, USEquityPricing.low])

### Further Example
Let's take another example where we build a [momentum](http://www.investopedia.com/terms/m/momentum.asp) custom factor and use it to create a filter. We will then use that filter as a `screen` for our pipeline.

Let's start by defining a `Momentum` factor to be the division of the most recent close price by the close price from `n` days ago where `n` is the `window_length`.

In [9]:
class Momentum(CustomFactor):
    # Default inputs
    inputs = [USEquityPricing.close]

    # Compute momentum
    def compute(self, today, assets, out, close):
        out[:] = close[-1] / close[0]

Now, let's instantiate our `Momentum` factor (twice) to create a 10-day momentum factor and a 20-day momentum factor. Let's also create a `positive_momentum` filter returning `True` for securities with both a positive 10-day momentum and a positive 20-day momentum.

In [10]:
ten_day_momentum = Momentum(window_length=10)
twenty_day_momentum = Momentum(window_length=20)

positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

Next, let's add our momentum factors and our `positive_momentum` filter to `make_pipeline`. Let's also pass `positive_momentum` as a `screen` to our pipeline.

In [11]:
def make_pipeline():

    ten_day_momentum = Momentum(window_length=10)
    twenty_day_momentum = Momentum(window_length=20)

    positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev,
            'ten_day_momentum': ten_day_momentum,
            'twenty_day_momentum': twenty_day_momentum
        },
        screen=positive_momentum
    )

Running this pipeline outputs the standard deviation and each of our momentum computations for securities with positive 10-day and 20-day momentum.

In [12]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

C:\Users\liude\Anaconda3\envs\bt36\lib\site-packages\numpy\lib\nanfunctions.py:1427: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


std_dev  ten_day_momentum  \
2015-05-05 00:00:00+00:00 Equity(000001 [平安银行])   0.110707          1.013497   
                          Equity(000002 [万 科Ａ])   0.203234          1.023927   
                          Equity(000006 [深振业Ａ])   0.275899          1.061395   
                          Equity(000007 [全新好])    1.047924          1.272638   
                          Equity(000009 [中国宝安])   0.986773          1.250419   
                          Equity(000012 [南 玻Ａ])   0.474864          1.422263   
                          Equity(000016 [深康佳Ａ])   1.181333          1.106096   
                          Equity(000019 [深深宝Ａ])   0.608033          1.287244   
                          Equity(000020 [深华发Ａ])   0.477088          1.109677   
                          Equity(000021 [深科技])    0.306425          1.146341   
                          Equity(000026 [飞亚达Ａ])   0.285475          1.054321   
                          Equity(000027 [深圳能源])   0.284422          1.072792   
                          Equity(000028 [国药一致])   2.044323          1.071560   
                          Equity(000029 [深深房Ａ])   0.286175          1.051259   
                          Equity(000030 [富奥股份])   0.331337          1.080134   
                          Equity(000031 [中粮地产])   0.315569          1.020779   
                          Equity(000032 [深桑达Ａ])   0.467547          1.067410   
                          Equity(000033 [新都退])    0.385019          1.408412   
                          Equity(000037 [深南电A])   0.364999          1.080000   
                          Equity(000042 [中洲控股])   0.620084          1.041163   
                          Equity(000043 [中航地产])   0.203568          1.007335   
                          Equity(000045 [深纺织Ａ])   1.237730          1.342972   
                          Equity(000046 [泛海控股])   0.608704          1.097214   
                          Equity(000048 [康达尔])    1.352016          1.274885   
                          Equity(000055 [方大集团])   0.350599          1.128501   
                          Equity(000056 [皇庭国际])   0.704483          1.076047   
                          Equity(000058 [深 赛 格])  0.392479          1.126866   
                          Equity(000059 [华锦股份])   0.256484          1.077689   
                          Equity(000060 [中金岭南])   0.378809          1.087990   
                          Equity(000061 [农 产 品])  0.704170          1.131490   
...                                                    ...               ...   
                          Equity(603338 [浙江鼎力])   1.209354          1.183626   
                          Equity(603368 [柳州医药])   3.750778          1.119746   
                          Equity(603369 [今世缘])    0.465463          1.009822   
                          Equity(603456 [九洲药业])   0.918882          1.261832   
                          Equity(603518 [维格娜丝])   0.547189          1.084515   
                          Equity(603519 [立霸股份])   1.383237          1.171395   
                          Equity(603555 [贵人鸟])    1.875814          1.030548   
                          Equity(603558 [健盛集团])   0.999428          1.096689   
                          Equity(603588 [高能环境])   2.305744          1.162484   
                          Equity(603600 [永艺股份])   5.457015          1.716683   
                          Equity(603606 [东方电缆])   0.663644          1.064137   
                          Equity(603609 [禾丰牧业])   0.380557          1.092790   
                          Equity(603611 [诺力股份])   0.872935          1.053515   
                          Equity(603618 [杭电股份])   1.286568          1.188676   
                          Equity(603636 [南威软件])   1.549729          1.151787   
                          Equity(603678 [火炬电子])   1.942318          1.245938   
                          Equity(603686 [龙马环卫])   4.043309          1.276870   
                          Equity(603698 [航天工程])   0.704415          1.000516   
                          Equity(603699 [纽威股份])   0

Custom factors allow us to define custom computations in a pipeline. They are frequently the best way to perform computations on [partner datasets](https://www.zipline.com/data) or on multiple data columns. The full documentation for CustomFactors is available [here](https://www.zipline.com/help#custom-factors).

In the next lesson, we'll use everything we've learned so far to create a pipeline for an algorithm.